In [1]:
import os
import pandas as pd
import numpy as np
import re
import dask.dataframe as dd

# path = 'C:/Users/user/Desktop/AIML/datasets/Data_Preprocessing/ham_set'
path = '/home/maroco/dataset/ham_dataset'
files = os.listdir(path)
filenames = []

for root, dirs, files in os.walk(path):
  for file in files:
    if file.endswith('.csv'):
      filenames.append(os.path.join(root, file))

all_pre_dfs = []

# Iterate through the list of filenames and read each CSV file
for filename in filenames:
  pre_df = pd.read_csv(filename, sep='\t')
  pre_df.columns = ['text']
  pre_df['label'] = 'ham'

  # Create a list to store DataFrames
  dpre_df = dd.from_pandas(pre_df, npartitions=4) # Create dpre_df after modifications

  # Append the current DataFrame to the list
  result = dpre_df.compute()
  all_pre_dfs.append(result)

# Concatenate all DataFrames in the list into a single DataFrame
raw = pd.concat(all_pre_dfs, ignore_index=True)

/home/maroco/anaconda3/envs/kobert_env/lib/python3.10/site-packages/dask/dataframe/__init__.py:49: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
# 텍스트 처리 함수 설정
def preprocess_text(text):
    # 여러 개의 공백을 하나의 공백으로 축소
    text = re.sub(r'\s+', ' ', text)
    # 앞뒤 공백 제거
    text = text.strip()
    # 맨 앞의 숫자와 공백 또는 콜론 제거
    text = re.sub(r'^\d+\s*', '', text)
    # 필요에 따라 특수 문자 제거 (예시로 일부 문자만 제거)
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text)
    return text

In [3]:
# 결측치 처리
print(len(raw))
drop_index = raw[raw['text'].isnull()].index
raw = raw.drop(drop_index)
print(len(raw))

42457
42457


In [4]:
# 중복 내용 제거
print(len(raw))
raw = raw.drop_duplicates('text')
print(len(raw))

42457
40350


In [5]:
raw['text']=raw['text'].apply(preprocess_text)

In [6]:
raw.head()

,text,label
0,맞지 하하 중간에 있어서 하하,ham
1,기차타면 들리지 않음 키키,ham
2,대전은 참 교통이 편리해,ham
3,어디든 대전 통과 할듯 거의 하하,ham
4,맞음 대전은 중간이라 좋음,ham


In [7]:
# raw.to_csv('C:/Users/user/Desktop/AIML/datasets/Data_Preprocessing/ham_data.csv', index=False)
raw.to_csv('/home/maroco/dataset/preprocessed_set/ham_data.csv', index=False)